# Задание 1

In [75]:
import torch
import numpy as np

#####
1. PyTorch предоставляет структуру данных Tensor, которая очень похожа на Numpy nndaray.
2. Разбивает вычисления на части, которые считаются последовательно(представляется это ввиде динамического графа, где каждая вершина независимый код, обрабатывающий свою операцию). Все сложные вычисления происходят на С/С++
3. Каждый torch.Tensor содержит компоненты:
 3.1 grad (значение градиента) или
 3.2 grad_fn(ссылка на функцию вычисления градиента)

### autograd

##### Если для тензора нужно вычислить градиент, тогда requires_grad нужно присвоить значение True, по умолчанию False. Создается динамический граф, в котором автоматически создаются обратные функции, и при вызове .backward() запустится проход по графу в обратную сторону и сосчитается grad

In [78]:
x = torch.randn(3, requires_grad=True)
print(x)
y = x*5
print(y)

z = y.mean()
print(z)

z.backward()
print(x.grad)

tensor([-0.4618,  0.6820, -1.1559], requires_grad=True)
tensor([-2.3090,  3.4102, -5.7796], grad_fn=<MulBackward0>)
tensor(-1.5595, grad_fn=<MeanBackward0>)
tensor([1.6667, 1.6667, 1.6667])


#### Для вектора.

In [59]:
x = torch.ones(3, requires_grad=True)
print(x)
y = x*5
print(y)

g = y+4
print(g)

g.backward(torch.ones_like(g))
print(x.grad)

tensor([1., 1., 1.], requires_grad=True)
tensor([5., 5., 5.], grad_fn=<MulBackward0>)
tensor([9., 9., 9.], grad_fn=<AddBackward0>)
tensor([5., 5., 5.])


#### Также с помощью пакета autograd можно вычислять jacobian, hessian

In [99]:
from autograd import jacobian

def exp(x):
   return x.exp()

X = torch.rand(2, 2)
jacobian(exp, X)

tensor([[0.0068, 0.3230],
        [0.6541, 0.0238]])
tensor([[1.0068, 1.3812],
        [1.9234, 1.0241]])


### grad

In [111]:
# f(x) = x^2
a = (torch.tensor([-2., -1., 1., 4.]),)
b = torch.tensor([4., 1., 1., 16.], )
torch.gradient(b, spacing=a)

(tensor([-1.5000, -0.7500,  3.7500,  7.5000]),)

In [112]:
b = torch.tensor([2., 1., 1., 0.], )
torch.gradient(b, spacing=2)

(tensor([-0.5000, -0.2500, -0.2500, -0.5000]),)

### Autogradient for previous task

In [189]:
import numpy as np
from math import *
import matplotlib.pyplot as plt
from numpy import linalg as LA

# helpers
f_calls_counter = 0
grad_calls_counter = 0
cur_lr = 0

def get_min_by_f(f, eps, lr, x, start_lr = 0.1) :
    global cur_lr
    cur_lr = start_lr
    points = np.asarray([])
    counter = 0
    next = next_x(x, lr, f, counter)
    while LA.norm(next - x) > eps:
        counter += 1
        points = np.append(points, x)
        x = next
        next = next_x(x, lr, f, counter, eps)
    return [x, counter, points]

# multidimension function in point [x_0, ..., x_n]
def f(x):
    a = x[0]
    return a**2 + 5*a + 10  

def next_x(x, lr, f, epoch = 0, eps = 0.0001) : 
    global cur_lr 
    cur_lr = lr(cur_lr, epoch)
    return x - cur_lr * grad(x, f, eps)    

def grad(x, f, eps):
        derivative = np.zeros(np.size(x))
        for i in range(np.size(x)):
            x[i] += eps
            f1 = f(x)
            x[i] -= 2 * eps
            f2 = f(x)
            x[i] += eps
            derivative[i] = (f1 - f2) / (2 * eps)
        return derivative   

  


#### Numpy градиентный спуск

In [225]:
import time

    
start_time = time.time()    
answer = get_min_by_f(f, 1e-6, (lambda lr, epoch: 0.16), np.asarray([7.0]))
print(" %s seconds" % (time.time() - start_time))
print("epoch: ", answer[1])
print("min :", answer[0])   
print("f(x):", f(answer[0])) 

 0.002953052520751953 seconds
epoch:  39
min : [-2.49999721]
f(x): 3.750000000007783


#### Pytorch градиентный спуск

In [220]:
def torch_gd(f, eps, lr, x_, start_lr = 0.16, max_epoch=1000):
    points = np.zeros((max_epoch, 1))
    x = [x_]
    points[0] = x[0].detach().numpy()
    cur_lr = start_lr
    i=0
    while i<max_epoch-1:
        i += 1
        t = f(x[-1])
        t.backward()
        cur_lr = lr(cur_lr, 0)
        points[i] = x[-1].detach().numpy() - cur_lr * x[-1].grad.numpy()
        x.append(torch.tensor(points[i], requires_grad=True))
        if (LA.norm(points[i]-points[i-1])<eps): break
    return [x, i, points]  


In [224]:
start = time.time()    
answer = torch_gd(f, 1e-6, (lambda lr, epoch: 0.16), x_=torch.tensor([7.0], requires_grad=True))
print(" %s seconds" % (time.time() - start))
print("epoch: ", answer[1])
print("min :", answer[0][answer[1]])  
print("f(x):", f(answer[0][answer[1]]))

 0.014671087265014648 seconds
epoch:  40
min : tensor([-2.5000], dtype=torch.float64, requires_grad=True)
f(x): tensor(3.7500, dtype=torch.float64, grad_fn=<AddBackward0>)
